In [1]:
from pathlib import Path
from datetime import datetime
import re, csv

def ReadLength(file):
    output = []    
    outputtime = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
    seq = Path("sample.fastq")
    with open('output-' + outputtime + '.tsv','w') as tableOut:
        writer = csv.writer(tableOut, delimiter = "\t")
        writer.writerow(['Read_ID', 'Read_length'])
    
    with open(seq, 'r') as inputfile:
        lines = inputfile.read()
        for data in re.finditer (r'@(?P<Read_ID>.*)\n(?P<Read_seq>\w*)\n(?P<plus>.)\n(?P<Read_Qscore>.*)', lines):
            readID = (data.group('Read_ID'))
            readsequence = (data.group('Read_seq'))
            readlength = str(len(readsequence))
            data = (readID, readlength)
            outputlist = list(data)

            with open('output-' + outputtime + '.tsv','a') as tableOut:
                    writer = csv.writer(tableOut, delimiter = "\t")
                    for add in outputlist:
                        output.append(add)
                    writer.writerow(outputlist)
    
ReadLength("sample.fastq")
